In [1]:
using NGSIM
using AutomotiveDrivingModels
using AutoViz
using Interact # Make video in notebook
using Reel # Save video as gif
using CSV # For writing to csv
using DataFrames # For writing to csv
using Distributions
using Test
using StatsBase
using Dates
using Plots
using JLD

┌ Info: Recompiling stale cache file /home/raunak/.julia/compiled/v1.1/Interact/XmYW4.ji for Interact [c601a237-2ae4-5e1e-952c-7a85b0c7eef1]
└ @ Base loading.jl:1184


HTML{String}("<script>\n// Immediately-invoked-function-expression to avoid global variables.\n(function() {\n    var warning_div = document.getElementById(\"webio-warning-2340424835042862171\");\n    var hide = function () {\n        var script = document.getElementById(\"webio-setup-2800130764369096753\");\n        var parent = script && script.parentElement;\n        var grandparent = parent && parent.parentElement;\n        if (grandparent) {\n            grandparent.style.display = \"none\";\n        }\n        warning_div.style.display = \"none\";\n    };\n    if (typeof Jupyter !== \"undefined\") {\n        console.log(\"WebIO detected Jupyter notebook environment.\");\n        // Jupyter notebook.\n        var extensions = (\n            Jupyter\n            && Jupyter.notebook.config.data\n            && Jupyter.notebook.config.data.load_extensions\n        );\n        if (extensions && extensions[\"webio/main\"]) {\n            // Extension already loaded.\n            console.log(\"Jupyter WebIO nbextension detected; not loading ad-hoc.\");\n            hide();\n            return;\n        }\n        console.warn(\"Loading WebIO Jupyter extension on an ad-hoc basis. Consider enabling the WebIO nbextension for a stabler experience (this should happen automatically when building WebIO).\");\n        warning_div.innerHTML = \"<strong>Loading WebIO Jupyter extension on an ad-hoc basis. Consider enabling the WebIO nbextension for a stabler experience (this should happen automatically when building WebIO).</strong>\";\n    } else if (window.location.pathname.includes(\"/lab\")) {\n        // Guessing JupyterLa\n        console.log(\"Jupyter Lab detected; make sure the @webio/jupyter-lab-provider labextension is installed.\");\n        hide();\n        return;\n    }\n})();\n\n</script>\n<p\n    id=\"webio-warning-2340424835042862171\"\n    class=\"output_text output_stderr\"\n    style=\"padding: 1em; font-weight: bold;\"\n>\n    Unable to load WebIO. Please make sure WebIO works for your Jupyter client.\n    For troubleshooting, please see <a href=\"https://juliagizmos.github.io/WebIO.jl/latest/providers/ijulia/\">\n    the WebIO/IJulia documentation</a>.\n    <!-- TODO: link to installation docs. -->\n</p>\n")

┌ Info: Recompiling stale cache file /home/raunak/.julia/compiled/v1.1/CSV/HHBkp.ji for CSV [336ed68f-0bac-5ca0-87d4-7b16caf5d00b]
└ @ Base loading.jl:1184
┌ Info: Recompiling stale cache file /home/raunak/.julia/compiled/v1.1/Plots/ld3vC.ji for Plots [91a5bcdd-55d7-5caf-9e0b-520d859cae80]
└ @ Base loading.jl:1184


In [36]:
include("admin_functions.jl"); # Brings in the functions defined in admin_functions.jl

In [3]:
traj_ngsim = load_trajdata(1)
roadway_ngsim = ROADWAY_101;

# Utils

In [37]:
"""

# Caution
- `dt` defined within the function. Here is 0.1, highd had 0.04

# Example
```julia
td1 = load_trajdata(1);
scene = Scene(500)
get!(scene,td1,300)
display(scene.entities[2].state.posF.s)
roadway = ROADWAY_101;
particle = Dict(:v_des=>25.0,:σ=>0.5)
hallucinate_a_step(roadway,scene,particle,car_id=scene.entities[2].id)
```
"""
function hallucinate_a_step(roadway,scene_input,particle;car_id=-1)
    if car_id==-1 @show "Please give valid car_id" end
    
    scene = deepcopy(scene_input)
    #scene = scene_input # This was the failure case
    n_cars = scene.n 

    models = Dict{Int, DriverModel}()
    
    # Create driver models for all the cars in the scene
    for veh in scene
        if veh.id == car_id
            models[veh.id] = IntelligentDriverModel(;particle...)
        else
            # TODO: RESEARCH QUESTION: What drives the other vehicles in the hallucination
            models[veh.id] = IntelligentDriverModel(v_des=10.0)
        end
    end
    
    n_steps = 1
    dt = 0.1
    rec = SceneRecord(n_steps, dt)
    
    simulate!(rec, scene, roadway, models, n_steps)
    
    X = Array{Float64}(undef,n_steps, 1)

    for t in 1:n_steps
        f = rec.frames[n_steps - t + 1]
        
            # Access the vehicle with id as car_id and return its frenet s
        X[t,1] = scene.entities[findfirst(car_id,f)].state.posF.s
    end
    return X[1]
end

hallucinate_a_step

In [38]:
"""

# Caution
- `timestep` defined in the function. Was 0.04 for highd. Here is 0.1
"""
function compute_particle_likelihoods(roadway,f,trupos,p_set_dict;car_id=-1)
    if car_id==-1 @show "Please give valid car_id" end
    timestep = 0.1 #TODO: Remove hardcoding
    p_mat, params, vec_val_vec = to_matrix_form(p_set_dict)
    
    num_params=size(p_mat)[1]
    num_p = size(p_mat)[2]
    lkhd_vec = Array{Float64}(undef,num_p)
    for i in 1:num_p    
        # Create dict version for a single particle
        p_dict = Dict()
        for j in 1:num_params
            p_dict[params[j]]=vec_val_vec[j][i]
        end
        
        std_dev_acc = p_dict[:σ]
        
        # hack to avoid the std_dev_pos become negative and error Normal distb
        if std_dev_acc <= 0 std_dev_acc = 0.1 end
        
        # TODO: This math needs to be verified from random variable calculations
        std_dev_pos = timestep*timestep*std_dev_acc
            
        hpos = hallucinate_a_step(roadway,f,p_dict,car_id=car_id)
        lkhd_vec[i] = pdf(Normal(hpos,std_dev_pos),trupos[1])
    end
    return lkhd_vec,p_mat,params
end

compute_particle_likelihoods

In [39]:
"""
    function update_p_one_step(roadway,f,trupos,p_set_dict;car_id=-1)

New particle set based on one step of resampling
"""
function update_p_one_step(roadway,f,trupos,p_set_dict;car_id=-1)
    if car_id==-1 @show "Provide valid car_id" end
    
    lkhd_vec,p_mat,params = compute_particle_likelihoods(roadway,f,trupos,p_set_dict,
                                                        car_id=car_id)
    
    num_params = size(p_mat)[1]
    num_p = size(p_mat)[2]

    ll_vec = log.(lkhd_vec)
    
    # TODO: This is hacky. What if log is not negative
    for i in 1:num_p
        if isinf(ll_vec[i])
            ll_vec[i] = 0
        else
            ll_vec[i] = -1*1/ll_vec[i]
        end
    end
    
    p_weight_vec = weights(ll_vec./sum(ll_vec)) # Convert to weights form to use julia sampling
    idx = sample(1:num_p,p_weight_vec,num_p)
    new_p_mat = p_mat[:,idx] #Careful that idx is (size,1) and not (size,2)
    
    new_p_set_dict = to_dict_form(params,new_p_mat)
    return new_p_set_dict
end

update_p_one_step

In [40]:
"""
    function get_ngsim_scene(framenum::Int64)

Get a specific scene from traj_ngsim

# Used by
- `multistep_update`

# Example:
```julia
scene = get_ngsim_scene(1)
```
"""
function get_ngsim_scene(framenum::Int64)
    scene = Scene(500)
    get!(scene,traj_ngsim,framenum)
    return scene
end

get_ngsim_scene

In [41]:
"""
    function sample_init_particles(num_p::Int64;v=30.0)

Inspired from `gen_test_particles` in `admin.jl`.
Samples `num_p` particles neighbourhood around the input `v`

# Used by
- `multistep_update`

# Example
```julia
sample_init_particles(10,v=20.0)
```
"""
function sample_init_particles(num_p::Int64;v::Float64=30.0)
    v_particles = sample(v-5.0:0.5:v+5.0,num_p)
    sig_particles = sample(0.1:0.1:1.0,num_p)
    p_set_dict = Dict(:v_des=>v_particles,:σ=>sig_particles)
    return p_set_dict
end

sample_init_particles

In [42]:
"""
    function get_veh_info(scene;car_id = -1)

Get position and velocity of specific vehicle from scene

# Used by
- `multistep_update`

# Example
```julia
scene = get_highd_scene(7)
pos,vel = get_veh_info(scene,car_id = 6)
@show pos,vel
```
"""
function get_veh_info(scene;car_id = -1)
    @assert car_id>0
    pos = scene.entities[findfirst(car_id,scene)].state.posF.s
    vel = scene.entities[findfirst(car_id,scene)].state.v
    return pos,vel
end

get_veh_info

In [43]:
"""
    function plot_particles(p_set_mat::Array{Float64,2},time::Float64)

Plot the distribution of particles
# Caution 
Only work for 2 parameters. Not developed for 3 parameter case yet

# Used by
- `multistep_update`

# Example:
```julia
set_for_plotting = to_particleMatrix(p_set_new)
plots = []
push!(plots,plot_particles(set_for_plotting,framenum*0.04))
```
"""
function plot_particles(p_set_mat::Array{Float64,2},time::Float64)
    # Check that number of params does not exceed 3
    
    @assert size(p_set_mat,1) <= 3
    plt = Plots.plot()
    # 2 parameter case	
    if size(p_set_mat,1) == 2
        plt = scatter(p_set_mat[1,:],p_set_mat[2,:],
            leg=false,title="time=$(time)",xlim=(25,35),ylim=(0,1.2),
                xlabel="v_des(m/s)",ylabel="sigma")

    # 3 parameter case
    else
        plt = scatter(p_set_mat[1,:],p_set_mat[2,:],p_set_mat[3,:],leg=false)
        scatter!([true_params[1]],[true_params[2]],[true_params[3]])
    end
    return plt
end

plot_particles

In [44]:
"""
    function make_gif(plots;filename="output.mp4")
Make a video using an array of plots. Uses the Reel library

# Used by
- `multistep_update`
"""
function make_gif(plots;filename="output.mp4")
@assert typeof(filename) == String
#@show "Making gif"
    frames = Frames(MIME("image/png"), fps=10)
    for plt in plots
        push!(frames, plt)
    end
    write(string("media/",filename), frames)
    return nothing
end # End of the reel gif writing function

make_gif

In [45]:
"""
    function multistep_update(num_p::Int64,car_id::Int64,
        start_frame::Int64,last_frame::Int64;p_dist_vid=false)

Returns the mean particle after running multiple steps of filtering

# Arguments
- `start_frame`: Frame to start filtering from
- `last_frame`: Frame to end filtering at
- `num_p`: Number of particles

# Keyworded arguments
- `p_dist_vid`: Default false. Makes a video of particle distribution if true

# Returns
- `mean_particle`: Dict containing the mean values of the final particle set after filtering

# Example
```julia
# Make a video for particle evolution for vehicle 13 over 500 steps starting from
# frame 11 and ending at frame number 60
multistep_update(500,13,11,60,p_dist_vid=true)
```
"""
function multistep_update(num_p::Int64,car_id::Int64,
        start_frame::Int64,last_frame::Int64;p_dist_vid=false)

    startscene = get_ngsim_scene(start_frame)
    startpos,startvel = get_veh_info(startscene,car_id=car_id)

    p_set_dict = sample_init_particles(num_p,v=startvel)
    plots = []
    init_p_mat = to_particleMatrix(p_set_dict)
    push!(plots,plot_particles(init_p_mat,0.0))
    dt = 0.1 # Multiplies framenum in creating the plot

    for framenum in start_frame:last_frame
        #print("multistep says. Framenum = $(framenum)\n")
        scene = get_ngsim_scene(framenum+1)
        trupos,truvel = get_veh_info(scene,car_id=car_id)
        
        # Get the scene to start hallucinating from
        scene = get_ngsim_scene(framenum)
        p_set_new = update_p_one_step(roadway_ngsim,scene,trupos,p_set_dict,car_id=car_id)
        p_set_dict = p_set_new
        set_for_plotting = to_particleMatrix(p_set_new)
        push!(plots,plot_particles(set_for_plotting,framenum*dt))
    end

        # Find the mean particle after filtering is over
        # Defined in `admin.jl`
    mean_particle = find_mean_particle(p_set_dict)

        # Make a plot of the particle distribution evolution
    if p_dist_vid 
        print("Making a particle distribution video")
        date = Dates.now()
        make_gif(plots,filename = "vehid_$(car_id)_$(date).gif") 
    end
    
    return mean_particle
end

multistep_update

In [46]:
"""
    function obtain_driver_models(veh_id_list,num_particles,start_frame,last_frame)
Driver models for each vehicle in veh_id_list

# Arguments
- `veh_id_list` List with vehicle ids
- `start_frame` Frame to start filtering from
- `last_frame` Frame to end hallucination at

# Returns
- `models` Dict with veh id as key and IDM driver model as value
- `final_particles` Dict with veh id as key and avg particle over final particle set as value

# Example
```julia
veh_id_list = [72,75,73,67,69,71,64,59,56,57,62,60,54,55,49,51,48,43,47,39,37,34,44,33,31]
model_multimodels,particles_multimodels = obtain_driver_models(veh_id_list,500,300,375)
```
"""
function obtain_driver_models(veh_id_list,num_particles,start_frame,last_frame)
    models = Dict{Int64,DriverModel}() # key is vehicle id, value is driver model
    final_particles = Dict{Int64,Dict}() # key is vehicle id, value is final particles
        # Loop over all the cars and get their corresponding IDM parameters by particle filter
    for veh_id in veh_id_list
        print("obtain_driver_model says: vehicle id = $(veh_id)\n")
        mean_particle = multistep_update(num_particles,veh_id,start_frame,last_frame)
        final_particles[veh_id] = mean_particle
        models[veh_id] = IntelligentDriverModel(;mean_particle...)
    end
    return models,final_particles
end

obtain_driver_models

In [9]:
"""
    function keep_vehicle_subset!(scene::Scene, ids::Vector{Int})
Will come in useful to remove those vehicles from video who we do not learn filter for.
Obtained from `ngsim_env/julia/src/ngsim_utils.jl`

# Arguments
- `ids` List of integers with ids of vehicles to be kept
- `scene` A scene in which to keep vehicles given in `ids` and remove the rest of the vehicles

# Used by
- `run_simulation` Only keep those vehicles in the scene for which we have done filtering

# Example
```julia
scene = Scene(500)
get!(scene,traj_ngsim,300)
keep_vehicle_subset!(scene,veh_id_list)
```
"""
function keep_vehicle_subset!(scene::Scene, ids::Vector{Int})
    keep_ids = Set(ids)
    scene_ids = Set([veh.id for veh in scene])
    remove_ids = setdiff(scene_ids, keep_ids)
    for id in remove_ids
        delete!(scene, id)
    end
    return scene
end

keep_vehicle_subset!

In [47]:
"""
    function run_simulation(n_steps,models,start_frame,dt=0.1;ids_to_keep = [])
Run a simulation using driver models for specified number of timesteps

# Arguments
- `n_steps` Number of simulation steps
- `start_frame` Frame number to start hallucination from
- `models` Dictionary with veh id as key and corresponding driver model as value
- `dt` Timestep (default 0.1 for the ngsim dataset)
- `ids_to_keep` If not empty, then only vehicles present in `ids_to_keep` are retained

# Returns
- `rec` Scenerecord which contains the simulation result i.e. frames. CAUTION: Index backward

# Possible breakage
- `roadway_ngsim` is not passed in as an argument. Will need to make it global
- `model_1` hard coded here for the 50 step case starting from frame 10

# Example
```julia
rec = run_simulation(75,models,299,ids_to_keep = veh_id_list)
```
"""
function run_simulation(n_steps,models,start_frame,dt=0.1;ids_to_keep = [])
    scene = Scene(500)
    get!(scene,traj_ngsim,start_frame)
    
    if !isempty(ids_to_keep)
        keep_vehicle_subset!(scene,ids_to_keep)
    end

    rec = SceneRecord(n_steps,dt)
    simulate!(rec,scene,roadway_ngsim,models,n_steps)
    return rec
end

run_simulation

In [4]:
"""
    my_overlay
Overlaying hallucinated trajectory on the ground truth
# Fields
- `color::Colorant`
- `scene::Scene`
"""
# Borrowed from visualization_functions.jl
struct my_overlay <: SceneOverlay
    scene::Scene
    color # Needs to be of form colorant"Colorname"
end

function AutoViz.render!(rendermodel::RenderModel,overlay::my_overlay, 
        scene::Scene, roadway::Roadway)
    AutoViz.render!(rendermodel,overlay.scene,car_color = overlay.color)
    return rendermodel
end

In [5]:
"""
    IDOverlay
Display the ID on top of each entity in a scene.
# Fields
- `color::Colorant`
- `font_size::Int64`
"""
mutable struct IDOverlay <: SceneOverlay
    color::Colorant
    font_size::Int
end

function AutoViz.render!(rendermodel::RenderModel, overlay::IDOverlay, scene::Scene, 
                            env::E) where E
    font_size = overlay.font_size
    for veh in scene
        add_instruction!(rendermodel, render_text, ("$(veh.id)", veh.state.posG.x, 
                        veh.state.posG.y, font_size, overlay.color), incameraframe=true)
    end
    return rendermodel
end

In [48]:
"""
    function make_video(;range=nothing,filename="output.mp4")

Makes a video of the ngsim dataset on the ngsim roadway taking frame range as input

# Overlays
- Id number of all vehicles
- Timestep as a text overlay

# Example 
```julia
make_video(range=11:60,filename="ngsim_11to60.mp4")
```
"""
function make_video(;range=nothing,filename="ngsim_$(range).mp4")

    frames = Frames(MIME("image/png"), fps=5)
    scene = Scene(500)
    step = 0
    for i in range
        step= step+1
        temp_scene = get!(scene,traj_ngsim,i)
        
        carcolors = Dict{Int,Colorant}()
        for veh in temp_scene
            # Color cars green when they are in segment 3
            if veh.state.posF.roadind.tag.segment == 3
                carcolors[veh.id] = colorant"green"
            end
        end
        
        scene_visual = render(temp_scene, roadway_ngsim, 
            [IDOverlay(colorant"white",10),TextOverlay(text=["step=$(step)"],font_size=12)],
            cam=CarFollowCamera{Int}(51,2.0),
            #cam=StaticCamera(VecE2(1966600, 570750), 1.4),
            #cam=FitToContentCamera(-0.6),
            car_colors = carcolors
        )
        push!(frames,scene_visual)
    end
    write(string("media/"*filename),frames)    
    
    return nothing
end

make_video

In [34]:
"""
    function make_vid_truth_halluc(start_frame,last_frame,rec;filename="output.mp4",ids_to_keep = [])
Video with hallucination overlayed over ground truth trajectory

# Arguments
- `start_frame` Frame number to start from
- `last_frame` Frame number to end at
- `rec` SceneRecord which contains hallucinated trajectory
- `ids_to_keep` Id not empty list, vehicles in this list will be the only ones in the scene

# Overlays used
- `my_overlay` To allow overlaying of hallucination on top of ground truth
- `IDOverlay` Overlays the vehicle ids on the video

# Possible breakages
- `traj_highd` We will need to provide this maybe as a global variable
- `roadway_highd` Same. We need global variable

# Example
```julia
make_vid_truth_halluc(300,375,rec_multimodel,filename="ngsim_halluc.mp4",ids_to_keep = veh_id_list)
```
"""
function make_vid_truth_halluc(start_frame,last_frame,rec;filename="output.mp4",ids_to_keep = [])
    frames = Frames(MIME("image/png"), fps=10)
    scene = Scene(500)

    step = 0
    for i in start_frame:last_frame
        step = step+1
        temp_scene = get!(scene,traj_ngsim,i)
        if !isempty(ids_to_keep)
            keep_vehicle_subset!(temp_scene,ids_to_keep)
        end
        
        my_scene = rec[i-last_frame]
        if !isempty(ids_to_keep)
            keep_vehicle_subset!(my_scene,ids_to_keep)
        end
        
        overlay_hallucination = my_overlay(my_scene,colorant"white")
        #overlay_hallucination = my_overlay(rec[i-last_frame],colorant"white")
        
        scene_visual = render(temp_scene, roadway_ngsim, 
            [overlay_hallucination,IDOverlay(colorant"white",12),
                TextOverlay(text=["step=$step"],font_size = 14)],
            #cam=CarFollowCamera{Int}(2,5.0),
            #cam=StaticCamera(VecE2(1966400, 570900), 5.0),
            cam=FitToContentCamera(-1.5))
        push!(frames,scene_visual)
        
    end
    write(string("media/"*filename),frames)
    return nothing
end

make_vid_truth_halluc

In [58]:
"""
    function make_vid_truth_gail_idm
Make a video with ground truth, gail driven and idm driver cars all in one

# Example
```julia
make_vid_truth_gail_idm(300,400,env_rec,rec_multimodels,filename="all3.mp4",ids_to_keep = veh_id_list)
```
"""
function make_vid_truth_gail_idm(start_frame,last_frame,gail_rec,idm_rec;filename="output.mp4",ids_to_keep = [])
    frames = Frames(MIME("image/png"), fps=7)
    scene = Scene(500)

    step = 0
    for i in start_frame:last_frame
        step = step+1
        true_scene = get!(scene,traj_ngsim,i)
        if !isempty(ids_to_keep)
            keep_vehicle_subset!(true_scene,ids_to_keep)
        end
        
        gail_scene = gail_rec[i-last_frame]
        if !isempty(ids_to_keep)
            keep_vehicle_subset!(gail_scene,ids_to_keep)
        end
        
        idm_scene = idm_rec[i-last_frame]
        if !isempty(ids_to_keep)
            keep_vehicle_subset!(idm_scene,ids_to_keep)
        end
        
        gail_overlay = my_overlay(gail_scene,colorant"blue")
        idm_overlay = my_overlay(idm_scene,colorant"yellow")
        
        scene_visual = render(true_scene, roadway_ngsim, 
            [gail_overlay,idm_overlay,IDOverlay(colorant"white",12),
                TextOverlay(text=["step=$step"],font_size = 14)],
            #cam=CarFollowCamera{Int}(2,5.0),
            #cam=StaticCamera(VecE2(1966400, 570900), 5.0),
            cam=FitToContentCamera(-1.5))
        push!(frames,scene_visual)
        
    end
    write(string("media/"*filename),frames)
    return nothing
end

make_vid_truth_gail_idm

# Workspace

In [51]:
veh_id_list = [72,75,73,67,69,71,64,59,56,57,62,60,54,55,51,48,43,47,39,37,34,33]
    # Let's debug the bounds error issue
# veh_id_list = [31,44,49]
model_multimodels,particles_multimodels = obtain_driver_models(veh_id_list,500,300,375)

obtain_driver_model says: vehicle id = 72
obtain_driver_model says: vehicle id = 75
obtain_driver_model says: vehicle id = 73
obtain_driver_model says: vehicle id = 67
obtain_driver_model says: vehicle id = 69
obtain_driver_model says: vehicle id = 71
obtain_driver_model says: vehicle id = 64
obtain_driver_model says: vehicle id = 59
obtain_driver_model says: vehicle id = 56
obtain_driver_model says: vehicle id = 57
obtain_driver_model says: vehicle id = 62
obtain_driver_model says: vehicle id = 60
obtain_driver_model says: vehicle id = 54
obtain_driver_model says: vehicle id = 55
obtain_driver_model says: vehicle id = 51
obtain_driver_model says: vehicle id = 48
obtain_driver_model says: vehicle id = 43
obtain_driver_model says: vehicle id = 47
obtain_driver_model says: vehicle id = 39
obtain_driver_model says: vehicle id = 37
obtain_driver_model says: vehicle id = 34
obtain_driver_model says: vehicle id = 33


(Dict{Int64,DriverModel}(54=>IntelligentDriverModel(NaN, 0.9, 1.0, 4.0, 1.5, 14.2153, 5.0, 3.0, 2.0, 9.0),47=>IntelligentDriverModel(NaN, 1.0, 1.0, 4.0, 1.5, 11.5639, 5.0, 3.0, 2.0, 9.0),69=>IntelligentDriverModel(NaN, 0.9, 1.0, 4.0, 1.5, 9.52681, 5.0, 3.0, 2.0, 9.0),62=>IntelligentDriverModel(NaN, 0.9, 1.0, 4.0, 1.5, 14.7603, 5.0, 3.0, 2.0, 9.0),39=>IntelligentDriverModel(NaN, 1.0, 1.0, 4.0, 1.5, 13.1108, 5.0, 3.0, 2.0, 9.0),43=>IntelligentDriverModel(NaN, 1.0, 1.0, 4.0, 1.5, 20.1009, 5.0, 3.0, 2.0, 9.0),75=>IntelligentDriverModel(NaN, 1.0, 1.0, 4.0, 1.5, 10.6103, 5.0, 3.0, 2.0, 9.0),60=>IntelligentDriverModel(NaN, 1.0, 1.0, 4.0, 1.5, 13.4631, 5.0, 3.0, 2.0, 9.0),55=>IntelligentDriverModel(NaN, 1.0, 1.0, 4.0, 1.5, 16.5902, 5.0, 3.0, 2.0, 9.0),34=>IntelligentDriverModel(NaN, 0.9, 1.0, 4.0, 1.5, 14.2607, 5.0, 3.0, 2.0, 9.0)…), Dict{Int64,Dict}(54=>Dict{Any,Any}(:v_des=>14.2153,:σ=>0.9),47=>Dict{Any,Any}(:v_des=>11.5639,:σ=>1.0),69=>Dict{Any,Any}(:v_des=>9.52681,:σ=>0.9),62=>Dict{Any,Any

In [69]:
rec_multimodels = run_simulation(100,model_multimodels,299,ids_to_keep=veh_id_list)

SceneRecord(nscenes=100)

In [ ]:
make_vid_truth_halluc(301,375,rec_multimodels,filename="ngsim_halluc.mp4",ids_to_keep = veh_id_list)

In [70]:
make_vid_truth_gail_idm(301,400,rec_barrier,rec_multimodels,filename="all3.mp4",ids_to_keep = veh_id_list)

In [68]:
model_multimodels

Dict{Int64,DriverModel} with 22 entries:
  54 => IntelligentDriverModel(-0.104399, 0.9, 1.0, 4.0, 1.5, 14.2153, 5.0, 3.0…
  47 => IntelligentDriverModel(-0.235967, 1.0, 1.0, 4.0, 1.5, 11.5639, 5.0, 3.0…
  69 => IntelligentDriverModel(0.0682286, 0.9, 1.0, 4.0, 1.5, 9.52681, 5.0, 3.0…
  62 => IntelligentDriverModel(0.567901, 0.9, 1.0, 4.0, 1.5, 14.7603, 5.0, 3.0,…
  39 => IntelligentDriverModel(0.10013, 1.0, 1.0, 4.0, 1.5, 0.0, 5.0, 3.0, 2.0,…
  43 => IntelligentDriverModel(0.059199, 1.0, 1.0, 4.0, 1.5, 20.1009, 5.0, 3.0,…
  75 => IntelligentDriverModel(0.337163, 1.0, 1.0, 4.0, 1.5, 10.6103, 5.0, 3.0,…
  60 => IntelligentDriverModel(0.260351, 1.0, 1.0, 4.0, 1.5, 13.4631, 5.0, 3.0,…
  55 => IntelligentDriverModel(0.634789, 1.0, 1.0, 4.0, 1.5, 16.5902, 5.0, 3.0,…
  34 => IntelligentDriverModel(0.0982114, 0.9, 1.0, 4.0, 1.5, 14.2607, 5.0, 3.0…
  64 => IntelligentDriverModel(0.123388, 0.9, 1.0, 4.0, 1.5, 18.238, 5.0, 3.0, …
  59 => IntelligentDriverModel(0.1198, 0.8, 1.0, 4.0, 1.5, 14.3097, 

# Scripts

In [ ]:
# Just make a video of replay trajectories. Saves video to media folder
make_video(range=300:400)

In [ ]:
# Load gail driven scenerecord from jld and overlay on top of ground truth
rec_gail = JLD.load("gail_rec.jld","env_rec")
veh_id_list = [72,75,73,67,69,71,64,59,56,57,62,60,54,55,51,48,43,47,39,37,34,33]
make_vid_truth_halluc(301,400,rec_gail,
    filename = "gail_halluc_removed.mp4",ids_to_keep = veh_id_list)

In [62]:
# Barrier with gail
rec_barrier = JLD.load("barrier_rec.jld","env_rec")
veh_id_list = [72,75,73,67,69,71,64,59,56,57,62,60,54,55,51,48,43,47,39,37,34,33]
make_vid_truth_halluc(301,400,rec_barrier,filename = "gail_barrier_id39_id51.mp4",
    ids_to_keep = veh_id_list)

In [ ]:
# The idea of a barrier. Select some of the vehicles to be driven by IDM with v_des =0
# In this case, vehicle ids 39 and 51. Drive the rest of the vehicles in the list
# using driver models (gail/pf learned idm)
model_multimodels[39].v_des = 0
model_multimodels[51].v_des = 0
rec_multimodels = run_simulation(100,model_multimodels,299,ids_to_keep=veh_id_list)
make_vid_truth_gail_idm(301,400,rec_barrier,rec_multimodels,
    filename="all3.mp4",ids_to_keep = veh_id_list)